# Test Interspeech

In [4]:
PATH_TRANSCRIPTS = '/export/data_gpm/empatia/databases/MSP-podcast/Transcripts'

In [5]:
import os

def imprimir_contenido_txt(path, n):
    # Verificar si el path existe
    if not os.path.exists(path):
        print(f"El path {path} no existe.")
        return
    
    # Listar archivos en el directorio que terminan con .txt
    archivos = [f for f in os.listdir(path) if f.endswith('.txt')]
    
    # Si hay menos archivos que los solicitados, solo muestra los disponibles
    archivos_a_mostrar = archivos[:n]
    
    if not archivos_a_mostrar:
        print("No se encontraron archivos .txt.")
    else:
        for archivo in archivos_a_mostrar:
            print(f"\nContenido del archivo {archivo}:")
            archivo_path = os.path.join(path, archivo)
            try:
                with open(archivo_path, 'r') as f:
                    print(f.read())  # Imprime el contenido del archivo
            except Exception as e:
                print(f"Error al leer el archivo {archivo}: {e}")

imprimir_contenido_txt(PATH_TRANSCRIPTS, 5)



Contenido del archivo MSP-PODCAST_2432_0200.txt:
... happening there as well. and you have the black shirts and the brown shirts where the police either turned a blind eye or actively supported them, usually when they went on these rampages-

Contenido del archivo MSP-PODCAST_0133_0033.txt:
 look they're - they're reporting on absolute admitted facts. ha-ha." 

Contenido del archivo MSP-PODCAST_0288_0019.txt:
mr. [excess 00:01:24] also known as ike, bbc radio. download the bbc app. listen, share, and comment and whatever else [peel 00:01:29] says.

Contenido del archivo MSP-PODCAST_2546_0333_0003.txt:
and instead of us just handing people masks...

Contenido del archivo MSP-PODCAST_3820_0101_0000.txt:
or you're just done with all of the stuff that you've been doing, like system tuning.


## Data

In [6]:
import os
import pandas as pd
from tqdm import tqdm  # Importamos tqdm para la barra de progreso

def crear_dataframe_txt(path, output_csv):
    # Verificar si el path existe
    if not os.path.exists(path):
        print(f"El path {path} no existe.")
        return None
    
    # Listar archivos en el directorio que terminan con .txt
    archivos = [f for f in os.listdir(path) if f.endswith('.txt')]
    
    if not archivos:
        print("No se encontraron archivos .txt.")
        return None
    
    # Crear listas para almacenar los datos
    nombres = []
    textos = []
    
    # Recorrer los archivos .txt y leer su contenido con una barra de progreso
    for i, archivo in enumerate(tqdm(archivos, desc="Procesando archivos", unit="archivo")):
        archivo_path = os.path.join(path, archivo)
        try:
            with open(archivo_path, 'r') as f:
                nombres.append(archivo)
                textos.append(f.read())  # Almacena el contenido del archivo
        except Exception as e:
            print(f"Error al leer el archivo {archivo}: {e}")
    
    # Crear el DataFrame
    df = pd.DataFrame({
        'FileName': nombres,
        'text': textos
    })
    
    # Determinar el directorio del notebook y construir la ruta de salida
    output_path = os.path.join(os.getcwd(), output_csv)

    # Exportar el DataFrame a un archivo CSV
    try:
        df.to_csv(output_path, index=False)
        print(f"Archivo CSV exportado exitosamente a: {output_path}")
    except Exception as e:
        print(f"Error al exportar el archivo CSV: {e}")
    
    return df


df_txt = crear_dataframe_txt(PATH_TRANSCRIPTS, 'transcripts.csv')

if df_txt is not None:
    print(df_txt.head())  # Muestra las primeras filas del DataFrame


Procesando archivos:   1%|          | 939/116221 [00:09<20:14, 94.89archivo/s] 


KeyboardInterrupt: 

In [9]:
# Cargar el primer CSV (el que tiene los archivos .txt)
df_txt = pd.read_csv('data/transcripts.csv')

# Cargar el segundo CSV (el que tiene los archivos .wav)
# df_wav = pd.read_csv('/export/data_gpm/empatia/databases/MSP-podcast/Labels/labels_consensus.csv')
# df_wav = pd.read_csv('data/reduced_pseudo_female.csv')
df_wav = pd.read_csv('data/reduced_pseudo_male.csv')

# Mostrar las primeras filas de ambos DataFrames para ver la estructura
print("DataFrame con .txt:")
print(df_txt.head())

print("DataFrame con .wav:")
print(df_wav.head())

# Eliminar la extensión '.txt' en el DataFrame con los archivos .txt
df_txt['FileName'] = df_txt['FileName'].apply(lambda x: x.replace('.txt', ''))

# Eliminar la extensión '.wav' en el DataFrame con los archivos .wav
df_wav['FileName'] = df_wav['FileName'].apply(lambda x: x.replace('.wav', ''))

# Realizar el merge de ambos DataFrames en la columna 'FileName'
df_combinado = pd.merge(df_txt, df_wav, on='FileName', how='inner')  # 'inner' para solo coincidencias

# Mostrar las primeras filas del DataFrame combinado
print("DataFrame combinado:")
print(df_combinado.head())

# Exportar el DataFrame combinado a un archivo CSV
# df_combinado.to_csv('data.csv', index=False)
# df_combinado.to_csv('data/data_female.csv', index=False)
df_combinado.to_csv('data/data_male.csv', index=False)

DataFrame con .txt:
                         FileName  \
0       MSP-PODCAST_2432_0200.txt   
1       MSP-PODCAST_0133_0033.txt   
2       MSP-PODCAST_0288_0019.txt   
3  MSP-PODCAST_2546_0333_0003.txt   
4  MSP-PODCAST_3820_0101_0000.txt   

                                                text  
0  ... happening there as well. and you have the ...  
1   look they're - they're reporting on absolute ...  
2  mr. [excess 00:01:24] also known as ike, bbc r...  
3     and instead of us just handing people masks...  
4  or you're just done with all of the stuff that...  
DataFrame con .wav:
                         FileName EmoClass  EmoAct  EmoVal  EmoDom  SpkrID  \
0       MSP-PODCAST_0133_0554.wav        C     5.0     2.8     5.2      54   
1  MSP-PODCAST_1542_0059_0012.wav        N     3.2     4.0     4.8     804   
2       MSP-PODCAST_3117_0573.wav        N     3.2     3.2     4.4    1519   
3       MSP-PODCAST_2792_1031.wav        X     4.2     4.6     4.6     229   
4  MSP-PODCAST_13

## Gender

In [1]:
df_male = df_combinado[df_combinado['Gender'] == 'Male']
df_female = df_combinado[df_combinado['Gender'] == 'Female']

df_male.to_csv('data_male.csv', index=False)
df_female.to_csv('data_female.csv', index=False)

NameError: name 'df_combinado' is not defined